In [1]:
import numpy as np
import pandas as pd
import time
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib
import os
# %matplotlib notebook
%matplotlib inline

In [2]:
# body_part = "head"
# file_names = ['./data/training_data/head_data_1.log',
#              './data/training_data/head_data_2.log',
#              './data/training_data/head_data_3.log',
#              './data/training_data/head_data_4.log',
#              './data/training_data/head_data_5.log',
#              './data/training_data/head_data_6.log']

body_part = "shoulder_right"
file_names = ['./data/training_data/shoulder_right_data_3.log',
             './data/training_data/shoulder_right_data_4.log',
             './data/training_data/shoulder_right_data_5.log',
             './data/training_data/shoulder_right_data_6.log',
             './data/training_data/shoulder_right_data_7.log',
             './data/training_data/shoulder_right_data_8.log',
             './data/training_data/shoulder_right_data_9.log']

# body_part = "shoulder_left"
# file_names = ['./data/training_data/shoulder_left_data_1.log',
#               './data/training_data/shoulder_left_data_2.log',
#               './data/training_data/shoulder_left_data_3.log']

dataset = [pd.read_csv(f, delim_whitespace=True, header=0) for f in file_names]
dataset = pd.concat(dataset)

print('%d values'%(len(dataset)))
dataset = dataset[(np.abs(stats.zscore(dataset[["roll", "pitch", "yaw"]])) < 2.75).all(axis=1)]
# dataset = dataset[(np.abs(stats.zscore(dataset)) < 2).all(axis=1)]

590289 values


In [3]:
dataset = dataset.values[1:len(dataset)-1,0:]
# np.random.shuffle(dataset)

dataset = dataset[abs(dataset[:,12])!=0.0,:]
dataset = dataset[abs(dataset[:,13])!=0.0,:]
dataset = dataset[abs(dataset[:,14])!=0.0,:]
print('%d values after filtering outliers'%(len(dataset)))

euler_set = dataset[:, 12:15]
print('max euler ' + str(np.amax(euler_set)))
print('min euler ' + str(np.amin(euler_set)))
sensors_set = dataset[:, :12]

577888 values after filtering outliers
max euler 1.446942687034607
min euler -2.01660901877976


In [4]:
sin_cos_set = np.hstack([np.sin(euler_set), np.cos(euler_set)])

In [5]:
sensors_scaler = MinMaxScaler(feature_range=(-1., 1.))
sensors_set = sensors_scaler.fit_transform(sensors_set)

In [6]:
data_split = 0.8
split_idx = int(len(sensors_set)*data_split)

In [7]:
look_back = 10

data_in = []
data_out = []

for i in range(len(sensors_set)-look_back):
    data_in.append(sensors_set[i:i+look_back])
    data_out.append(sin_cos_set[i+1:i+look_back+1])

In [8]:
data_split = 0.8
data_idx = np.arange(len(data_in))
np.random.shuffle(data_idx)
split_idx = int(len(data_in)*data_split)

train_in = np.array(data_in)[data_idx[:split_idx]]
train_out = np.array(data_out)[data_idx[:split_idx]]
test_in = np.array(data_in)[data_idx[split_idx:]]
test_out = np.array(data_out)[data_idx[split_idx:]]

In [9]:
train_in.shape

(462302, 10, 12)

## Train model

In [10]:
from nn_model import NeuralNetworkModel, LSTMNeuralNetworkModel

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
model_path = './output/'+body_part+'_lstm'

if not os.path.exists(model_path):
    os.makedirs(model_path)

model_name = model_path+'/best_model'
joblib.dump(sensors_scaler, model_path+'/scaler.pkl') 

model = LSTMNeuralNetworkModel(name=body_part, hidden_size=100, look_back=look_back)
# model = NeuralNetworkModel(name=body_part, hidden_size=100)
model.fit(x=train_in, y=train_out, x_val=test_in, y_val=test_out, save_path=model_name, 
         iteration=500, patience=100, batch_size=1000)

/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


#0000: Traing loss 0.40340, Valid loss 0.08630
#0010: Traing loss 0.00590, Valid loss 0.00577
#0020: Traing loss 0.00334, Valid loss 0.00329
#0030: Traing loss 0.00264, Valid loss 0.00266
#0040: Traing loss 0.00231, Valid loss 0.00230
#0050: Traing loss 0.00210, Valid loss 0.00215
#0060: Traing loss 0.00196, Valid loss 0.00197
#0070: Traing loss 0.00183, Valid loss 0.00192
#0080: Traing loss 0.00174, Valid loss 0.00180
#0090: Traing loss 0.00168, Valid loss 0.00172
#0100: Traing loss 0.00162, Valid loss 0.00168
#0110: Traing loss 0.00156, Valid loss 0.00156
#0120: Traing loss 0.00151, Valid loss 0.00156
#0130: Traing loss 0.00146, Valid loss 0.00149
#0140: Traing loss 0.00142, Valid loss 0.00148
#0150: Traing loss 0.00138, Valid loss 0.00142
